In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import sounddevice as sd
import wavio
import tensorflow as tf
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical

# Dataset Paths
Crema_path = "Documents/Speech Emotion Recognition/Crema/"
Ravdess_path = "Documents/Speech Emotion Recognition/Ravdess/audio_speech_actors_01-24/"
Savee_path = "Documents/Speech Emotion Recognition/Savee/"
Tess_path = "Documents/Speech Emotion Recognition/Tess/"

# Emotion Mappings
CREMA_EMOTIONS = {"SAD": "sad", "ANG": "angry", "DIS": "disgust", "FEA": "fear", "HAP": "happy", "NEU": "neutral"}
RAVDESS_EMOTIONS = {1: "neutral", 2: "calm", 3: "happy", 4: "sad", 5: "angry", 6: "fearful", 7: "disgust", 8: "surprised"}
SAVEE_EMOTIONS = {"a": "angry", "d": "disgust", "f": "fear", "h": "happy", "n": "neutral", "sa": "sad", "su": "surprised"}

def process_crema():
    paths, emotions = [], []
    for file in os.listdir(Crema_path):
        if file.endswith(".wav"):
            parts = file.split("_")
            emotion = CREMA_EMOTIONS.get(parts[2], "Unknown")
            paths.append(os.path.join(Crema_path, file))
            emotions.append(emotion)
    return pd.DataFrame({"Path": paths, "Emotion": emotions})

def process_ravdess():
    paths, emotions = [], []
    for actor in os.listdir(Ravdess_path):
        actor_path = os.path.join(Ravdess_path, actor)
        if os.path.isdir(actor_path):
            for file in os.listdir(actor_path):
                if file.endswith(".wav"):
                    parts = file.split("-")
                    emotion = RAVDESS_EMOTIONS.get(int(parts[2]), "Unknown")
                    paths.append(os.path.join(actor_path, file))
                    emotions.append(emotion)
    return pd.DataFrame({"Path": paths, "Emotion": emotions})

def process_savee():
    paths, emotions = [], []
    for file in os.listdir(Savee_path):
        if file.endswith(".wav"):
            prefix = file[:2] if file[:2] in SAVEE_EMOTIONS else file[0]
            emotion = SAVEE_EMOTIONS.get(prefix, "Unknown")
            paths.append(os.path.join(Savee_path, file))
            emotions.append(emotion)
    return pd.DataFrame({"Path": paths, "Emotion": emotions})

def process_tess():
    paths, emotions = [], []
    for folder in os.listdir(Tess_path):
        folder_path = os.path.join(Tess_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".wav"):
                    parts = file.split("_")
                    emotion = "surprised" if parts[2] == "ps" else parts[2].lower()
                    paths.append(os.path.join(folder_path, file))
                    emotions.append(emotion)
    return pd.DataFrame({"Path": paths, "Emotion": emotions})

# Load and combine datasets
df_all = pd.concat([process_crema(), process_ravdess(), process_savee(), process_tess()], ignore_index=True)

def extract_features(file_path, max_pad_length=130):
    y, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

    if mfcc.shape[1] < max_pad_length:
        pad_width = max_pad_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode="constant")
    else:
        mfcc = mfcc[:, :max_pad_length]

    return mfcc.T

X = np.array([extract_features(path) for path in df_all["Path"]])
y = df_all["Emotion"]

#Encode Labels
encoder = LabelEncoder()
y_encoded = to_categorical(encoder.fit_transform(y))

# Reshape input for LSTM (samples, time steps, features)
X = np.array([x[:130, :] for x in X])
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)






In [15]:
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

In [ ]:
# LSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(130, 40))),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(y_encoded.shape[1], activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

#Save Model
model.save("lstm_emotion_model.h5")

Epoch 1/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 39s 111ms/step - accuracy: 0.3395 - loss: 1.8575 - val_accuracy: 0.5162 - val_loss: 1.2390
Epoch 2/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 50s 163ms/step - accuracy: 0.5299 - loss: 1.1921 - val_accuracy: 0.5750 - val_loss: 1.0938
Epoch 3/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 36s 117ms/step - accuracy: 0.5815 - loss: 1.0824 - val_accuracy: 0.5828 - val_loss: 1.0433
Epoch 4/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 35s 115ms/step - accuracy: 0.5891 - loss: 1.0571 - val_accuracy: 0.5746 - val_loss: 1.0508
Epoch 5/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 35s 114ms/step - accuracy: 0.6066 - loss: 1.0119 - val_accuracy: 0.5861 - val_loss: 1.0404
Epoch 6/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 36s 117ms/step - accuracy: 0.6131 - loss: 0.9829 - val_accuracy: 0.6149 - val_loss: 0.9780
Epoch 7/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 45s 147ms/step - accuracy: 0.6189 - loss: 0.9712 - val_accuracy: 0.6087 - val_loss: 0.9800
Epoch 8/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 35s 114ms/step - accuracy: 0.6412 - loss: 0

In [ ]:
# 🎙 Streamlit App
st.title("🎤 Real-Time Speech Emotion Recognition with LSTM")

# 📌 Function to Record Audio
def record_audio(duration=3, fs=44100):
    st.write("🎙 Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()
    wavio.write("recorded.wav", recording, fs, sampwidth=2)
    st.write("✅ Recorded Successfully!")
    return "recorded.wav"

# 📌 Predict Emotion
def predict_emotion(audio_path):
    features = extract_features(audio_path)
    features = np.expand_dims(features[:130, :], axis=0)  # Ensure shape matches model input
    prediction = model.predict(features)
    emotion = encoder.inverse_transform([np.argmax(prediction)])[0]
    return emotion

# 📌 Streamlit UI
if st.button("🎙 Record and Predict Emotion"):
    recorded_file = record_audio()
    predicted_emotion = predict_emotion(recorded_file)
    st.subheader(f"🎭 Detected Emotion: **{predicted_emotion}**")